In [ ]:
import sys
import os
import json

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

  
# append the path of the parent directory
sys.path.append("..")
from nonsmooth_implicit_diff import plot_utils
from nonsmooth_implicit_diff import utils

In [ ]:
def load_json_files(folder_path):
    # Iterate through all subfolders in the given folder
    exps = []
    for root, dirs, files in os.walk(folder_path):
        for subdir in dirs:
            subfolder_path = os.path.join(root, subdir)
            
            # Load results.json
            results_file_path = os.path.join(subfolder_path, 'results.json')
            if os.path.exists(results_file_path):
                with open(results_file_path, 'r') as results_file:
                    results = json.load(results_file)
                    print(f"Loaded results.json from {subfolder_path}: {results}")

            # Load config.json
            config_file_path = os.path.join(subfolder_path, 'conf.json')
            if os.path.exists(config_file_path):
                with open(config_file_path, 'r') as config_file:
                    config = json.load(config_file)
                    print(f"Loaded conf.json from {subfolder_path}: {config}")
            exps.append(dict(config=config, results=results))
    return exps

folder_path = '../exps/elastic_net_deterministic/'
exps = load_json_files(folder_path)

In [ ]:
# Sample data for the two line plots
for e in exps:
    config = e['config']
    results = utils.to_np(e['results'])
    hparams = dict(alpha_l1=config['alpha_l1'], alpha_l2=config['alpha_l2'], random_state=config['random_state'])
    print(hparams)
    
    t = results['hg_results']['fixed']['t']
    s_idx = results['hg_results']['fixed']['support_identification_index']
    
    lines_to_plot = (
        ('ITD', results['hg_results']['reverse'], 'solid'),
        ('AID-FP (k=t)', results['hg_results']['fixed'], 'dashed'),
        ('AID-CG (k=t)', results['hg_results']['CG'], 'dotted'),
    )
    
    metrics = (
        ('norm_diff', 'Approximation error'),
        ('norm_diff_norm', 'Normalized approx. error'),
    )
    

    for metric, metric_name in metrics:
        fig, ax = plt.subplots(figsize=(3, 3))
        
        ax.set_title(f"$\lambda_1= {hparams['alpha_l1']}$, $\lambda_2 = {hparams['alpha_l2']}$")

        if hparams['alpha_l2']== 0.0002:
            ax.set_xlim((-1,300))
        if hparams['alpha_l2']== 0.002 and hparams['alpha_l1']== 0.02:
            ax.set_xlim((-1,300))
        if hparams == {'alpha_l1': 0.001, 'alpha_l2': 0.1, 'random_state': 42}:
            ax.set_xlim((-1, 200))


        for (name, res, style) in lines_to_plot:
            line = res[metric]
            ax.plot(t, line, label=name, linestyle=style)

        ax.axvline(x=s_idx, linestyle='--', color='grey')
        ax.set_yscale('log')
        
        ax.set_xlabel('t')
        ax.set_ylabel(metric_name)
        


        ax.legend()
        plt.tight_layout()
        # plt.savefig(f'elastic_net_deterministic__al1_{hparams["alpha_l1"]}_al2_{hparams["alpha_l2"]}_seed_{hparams["random_state"]}_{metric}.pdf')
        plt.show()